In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Sahil GDrive
import numpy as np

train_1 = np.load(r'/content/drive/My Drive/Baby Cry/stratified k fold data/train_5.npy')
test_1 = np.load(r'/content/drive/My Drive/Baby Cry/stratified k fold data/test_5.npy')

train_lab_1 = np.load(r'/content/drive/My Drive/Baby Cry/stratified k fold data/train_lab_5.npy')
test_lab_1 = np.load(r'/content/drive/My Drive/Baby Cry/stratified k fold data/test_lab_5.npy')

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D,Flatten,Reshape,Cropping2D,Conv2DTranspose,concatenate,BatchNormalization,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.initializers import orthogonal
from tensorflow.keras.optimizers import Adam, SGD,RMSprop

input_file = Input(shape=(288,432, 3))

def Contract(x, filters, kernel , strides, padding, block_id, kernel_init=orthogonal()):
  prefix = f'block_{block_id}_'
  x = tf.keras.layers.Conv2D(filters= filters,kernel_size = kernel, strides = strides, kernel_initializer=kernel_init,
                             activation = None , padding = padding, name= prefix+'conv1_contract')(x)   
  x = BatchNormalization()(x)                                                   
  x = tf.keras.layers.Conv2D(filters= filters,kernel_size = kernel, strides = strides, kernel_initializer=kernel_init,
                             activation = None, padding = padding,  name= prefix+'conv2_contract')(x)
  x = tf.keras.layers.BatchNormalization(name=prefix+'conv_bn_contract')(x)
  x = tf.keras.layers.Dropout(0.2 ,name = prefix +'dropout_contract')(x)
  x = tf.keras.layers.LeakyReLU(name=prefix+'lrelu_contract')(x)
  return x

def Expand(x , y , filters, kernel, trans_kernel,strides, padding, block_id, kernel_init=orthogonal()):
  prefix = f'block_{block_id}_'
  x = tf.keras.layers.Conv2DTranspose(filters,trans_kernel,strides = strides,kernel_initializer = kernel_init, 
                                      padding = padding,name = prefix + 'trans_conv2d_expand')(x)
  x = BatchNormalization()(x)                                    
  x = tf.keras.layers.concatenate([x,y],name = prefix + 'concatenate_expand')
  x = tf.keras.layers.Conv2D(filters,kernel_size = kernel,activation = None ,kernel_initializer = kernel_init,
                             padding = padding,name = prefix + 'conv2d_1_expand')(x)
  x = BatchNormalization()(x)                           
  x = tf.keras.layers.Conv2D(filters,kernel_size=kernel,activation = None ,kernel_initializer = kernel_init,
                             padding = padding,name = prefix + 'conv2d_2_expand')(x)
  x = tf.keras.layers.BatchNormalization(name=prefix+'conv_bn_expand')(x)
  x = tf.keras.layers.LeakyReLU(name=prefix+'lrelu_expand')(x)
  x = tf.keras.layers.Dropout(0.2, name=prefix+'dropout_expand')((x))
  return x


def U_net_arch(input_shape):
  inputs = Input(shape=input_shape)
  
  #contracting path
  
  c1 = Contract(inputs, 32, 3 , strides = 1, padding='same', block_id=1)
  p1 = tf.keras.layers.MaxPooling2D((2,2),name = 'block_1_max_pool')(c1)

  c2 = Contract(p1, 64, 3 , strides = 1, padding='same', block_id=2)
  p2 = tf.keras.layers.MaxPooling2D((2,2),name = 'block_2_max_pool')(c2)

  c3 = Contract(p2, 128, 3 , strides = 1, padding='same', block_id=3)
  p3 = tf.keras.layers.MaxPooling2D((2,2),name = 'block_3_max_pool')(c3)

  c4 = Contract(p3, 256, 3 , strides = 1, padding='same', block_id=4)
  p4 = tf.keras.layers.MaxPooling2D((2,2),name = 'block_4_max_pool')(c4)

  c5 = Contract(p4, 512, 3 , strides = 1, padding='same', block_id=5)

  #Dense layers to reduce the bottleneck features
  
  x = Flatten(name = 'flatten_bottleneck')(c5)

  
  c5 = Reshape((18,27 , 512),name = 'reshape_flatten')(x)
  
  #expanding path
  
  c6 = Expand(c5, c4, 256, 3 , (2,2), strides = (2,2), padding='same', block_id = 6)
  
  c7 = Expand(c6, c3, 128, 3 , (2,2), strides = (2,2), padding='same', block_id = 7)
  
  c8 = Expand(c7, c2,64, 3 , (2,2), strides = (2,2), padding='same', block_id = 8)

  c9 = Expand(c8, c1, 32, 3 , (2,2), strides = (2,2), padding='same', block_id = 9)
  
  outputs = tf.keras.layers.Conv2D(3,(1,1),activation = 'sigmoid')(c9)
  
  return Model(inputs=inputs, outputs=outputs) 

input_shape = (288,432)
U_net = U_net_arch((*input_shape,3))

U_net.compile(optimizer=RMSprop(), loss='mean_squared_error') #msse

U_net.summary()






Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 288, 432, 3) 0                                            
__________________________________________________________________________________________________
block_1_conv1_contract (Conv2D) (None, 288, 432, 32) 896         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 288, 432, 32) 128         block_1_conv1_contract[0][0]     
__________________________________________________________________________________________________
block_1_conv2_contract (Conv2D) (None, 288, 432, 32) 9248        batch_normalization[0][0]        
_______________________________________________________________________________________

In [4]:
regularizer = tf.keras.regularizers.l2(.0001)
batch_size=20
for layer in U_net.layers:
    if layer.trainable == True:
        for attr in ['kernel_regularizer']:
            if hasattr(layer, attr):
                setattr(layer, attr, regularizer)


In [5]:
U_net.fit(train_1,train_1, batch_size=16,	validation_data=(test_1,test_1),	epochs=50, verbose=2, use_multiprocessing=True, workers=2)

Epoch 1/50
38/38 - 33s - loss: 0.0154 - val_loss: 0.0796
Epoch 2/50
38/38 - 29s - loss: 0.0039 - val_loss: 0.0223
Epoch 3/50
38/38 - 29s - loss: 0.0027 - val_loss: 0.0125
Epoch 4/50


KeyboardInterrupt: ignored

In [6]:
U_net.save_weights('U_net.h5')

In [7]:

for layer in U_net.layers:
  layer.trainable = False

In [8]:
from tensorflow.keras.layers import GlobalAveragePooling2D, AveragePooling2D
intermediate_output = U_net.get_layer("flatten_bottleneck").output
x = Reshape((18,27,512))(intermediate_output)
x = AveragePooling2D()(x)
x = Flatten()(x)

x = Dense(8192, activation="relu")(x)
x = Dense(4096, activation="relu")(x)
x = Dense(512, activation="relu",name = 'MLP_512')(x)
x = Dense(128, activation="relu",name = 'MLP_128')(x)   # we can experiment with more dense layers if output of average pooling layer is too big
x = Dropout(0.2,name = 'dropout_MLP')(x)
out_mlp = Dense(5, activation="softmax",name = 'output')(x)  # we have 5 classes

total_model = Model(inputs=U_net.input, outputs=out_mlp)
opt=Adam(lr=1e-4)
total_model.compile(loss='categorical_crossentropy', optimizer=SGD(),	metrics=["accuracy"])
total_model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 288, 432, 3)]     0         
_________________________________________________________________
block_1_conv1_contract (Conv (None, 288, 432, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 288, 432, 32)      128       
_________________________________________________________________
block_1_conv2_contract (Conv (None, 288, 432, 32)      9248      
_________________________________________________________________
block_1_conv_bn_contract (Ba (None, 288, 432, 32)      128       
_________________________________________________________________
block_1_dropout_contract (Dr (None, 288, 432, 32)      0         
_________________________________________________________________
block_1_lrelu_contract (Leak (None, 288, 432, 32)     

In [11]:
regularizer = tf.keras.regularizers.l2(.0001)
batch_size=20
for layer in total_model.layers:
    if layer.trainable == True:
        for attr in ['kernel_regularizer']:
            if hasattr(layer, attr):
                setattr(layer, attr, regularizer)

In [12]:
import tensorflow as tf
#callback for best Val Accuracy
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="/content/total_best.ckpt",
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
#total_model
total_model.fit(train_1, train_lab_1, batch_size=16,	validation_data=(test_1, test_lab_1),	epochs=50, verbose=2, callbacks=[model_checkpoint_callback])

Epoch 1/50
38/38 - 33s - loss: 1.3759 - accuracy: 0.4901 - val_loss: 1.4417 - val_accuracy: 0.5033
Epoch 2/50
38/38 - 4s - loss: 1.3892 - accuracy: 0.5017 - val_loss: 1.4007 - val_accuracy: 0.5033
Epoch 3/50
38/38 - 4s - loss: 1.3936 - accuracy: 0.5050 - val_loss: 1.4279 - val_accuracy: 0.5033
Epoch 4/50
38/38 - 4s - loss: 1.3713 - accuracy: 0.5050 - val_loss: 1.4237 - val_accuracy: 0.5033
Epoch 5/50
38/38 - 4s - loss: 1.3474 - accuracy: 0.5050 - val_loss: 1.4255 - val_accuracy: 0.5033
Epoch 6/50
38/38 - 4s - loss: 1.3526 - accuracy: 0.5050 - val_loss: 1.4863 - val_accuracy: 0.5033
Epoch 7/50
38/38 - 4s - loss: 1.3741 - accuracy: 0.5050 - val_loss: 1.4373 - val_accuracy: 0.5033
Epoch 8/50
38/38 - 4s - loss: 1.3568 - accuracy: 0.5050 - val_loss: 1.4339 - val_accuracy: 0.5033
Epoch 9/50
38/38 - 4s - loss: 1.3668 - accuracy: 0.5050 - val_loss: 1.4260 - val_accuracy: 0.5033
Epoch 10/50
38/38 - 4s - loss: 1.3554 - accuracy: 0.5050 - val_loss: 1.4218 - val_accuracy: 0.5033
Epoch 11/50
38/38 